# 自然语言识别处理单元
## [拂晓工作室](https://github.com/Errrneist/Alchemist)
* 此程序可用于识别文字中的语言以及深度分析新闻概况

# 参考资料
* [1] [Apple - SFrame实例](https://apple.github.io/turicreate/docs/api/generated/turicreate.SFrame.html#turicreate.SFrame)
* [2] [Apple - Turicreate中关于SFrame.apply()的文档](https://apple.github.io/turicreate/docs/api/generated/turicreate.SFrame.apply.html)
* [0] [Apple - Turicreate使用TrimRareWords来快速定义高频词](https://apple.github.io/turicreate/docs/api/generated/turicreate.text_analytics.trim_rare_words.html?highlight=remove%20punctuation)
* [0] [Apple - 用Turicreate生成Word Count Vector](https://apple.github.io/turicreate/docs/api/generated/turicreate.text_analytics.count_words.html)
* [0] [Apple - 关于WordProcessing以及使用dict_trim_by_values(n)删除频率以下的词以及TF-IDF的描述](https://turi.com/learn/userguide/text/analysis.html)
* [3] [Stackoverflow - Python从字符串中删除子字符串](https://stackoverflow.com/questions/31273642/better-way-to-remove-multiple-words-from-a-string)
* [4] [Stackoverflow - 关于SFrame.apply()和Lambda x的实例](https://stackoverflow.com/questions/33028423/how-can-i-use-apply-with-a-function-that-takes-multiple-inputs)
* [5] [楼宇 - 用Python分析《红楼梦》](https://zhuanlan.zhihu.com/p/29209681)
* [0] [zhon - hanzi文档](http://zhon.readthedocs.io/en/latest/)
* [6] [CSDN - 使用pyltp包进行中文分词实例](https://blog.csdn.net/sinat_26917383/article/details/77067515)
* [7] [CSDN - NLP+语义分析: 角色标注、篇章分析](https://blog.csdn.net/sinat_26917383/article/details/55683599)
* [8] [CSDN - NLP情感分析简介](https://blog.csdn.net/android_ruben/article/details/78174172)
* [9] [CSDN - 基于机器学习的NLP情感分析（二）—- 分类问题](https://blog.csdn.net/stary_yan/article/details/75330729)
* [0] [CSDN - 关于使用zhon.hanzi移除标点符号的解决方法](https://www.cnblogs.com/arkenstone/p/6092255.html)
* [0] [CSDN - Python中正则表达式sub函数用法总结](https://blog.csdn.net/hzliyaya/article/details/52495150)
* [0] [CSDN - Python中另一种移除标点符号的办法](https://blog.csdn.net/mach_learn/article/details/41744487)
* [10] [PYLTP - pyltp技术文档](http://pyltp.readthedocs.io/zh_CN/latest/api.html#id15)
* [11] [PYLTP - pyltp介绍文档一](https://www.ltp-cloud.com/intro/#introduction )
* [12] [PYLTP - pyltp介绍文档二](http://ltp.readthedocs.io/zh_CN/latest/appendix.html#id5)
* [13] [PYLTP - pyltp深度训练模型](https://pan.baidu.com/share/link?shareid=1988562907&uk=2738088569#list/path=/)


# 导入库

In [32]:
# 导入基本库
import re
import os
import time
import random
import datetime

# 网络获取相关包
import pymysql
import urllib
from bs4 import BeautifulSoup

# 机器学习与大数据框架
import turicreate as tc
import csv
from zhon.hanzi import punctuation

# 自然语言识别框架
import pyltp
from pyltp import SentenceSplitter
from pyltp import Segmentor
from pyltp import Postagger
from pyltp import NamedEntityRecognizer
from pyltp import Parser
LTP_DATA_DIR = '../../../LTP_data_v3.4.0/'  # ltp模型目录的路径
pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')  # 词性标注模型路径，模型名称为`pos.model`
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`
ner_model_path = os.path.join(LTP_DATA_DIR, 'ner.model')  # 命名实体识别模型路径，模型名称为`pos.model`
par_model_path = os.path.join(LTP_DATA_DIR, 'parser.model')  # 依存句法分析模型路径，模型名称为`parser.model`
srl_model_path = os.path.join(LTP_DATA_DIR, 'srl')  # 语义角色标注模型目录路径，模型目录为`srl`。注意该模型路径是一个目录，而不是一个文件。

# 载入数据

In [ ]:
# 用于从上次停止工作后恢复数据
news_path = '../DataSets/Eastmoney/News_NLP/China/'
news_source = 'CHINA20180702-1906.csv'
news = tc.SFrame(news_path + news_source)

In [4]:
# 载入数据
news_path = '../DataSets/Eastmoney/News/China/'
news_source = 'CHINA20180702-1906.csv'
news = tc.SFrame(news_path + news_source)

# 预览
news

Finished parsing file /Users/hongjunwu/Documents/GitHub/Alchemist/DataSets/Eastmoney/News/China/CHINA20180702-1906.csv

Parsing completed. Parsed 100 lines in 0.029413 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,int,list,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/hongjunwu/Documents/GitHub/Alchemist/DataSets/Eastmoney/News/China/CHINA20180702-1906.csv

Parsing completed. Parsed 509 lines in 0.041584 secs.

contents,date,day,month,related
小微企业融资难、融资贵问题，是一个世界性难题。甚至，外延更为丰富的普惠金融，在很多时候、在不 ...,20180630,30,6,"[保险, 陆家嘴, 银行]"
据国家统计局30日消息，2018年6月份，中国制造业采购经理指数(PMI)为51.5%，比上 ...,20180630,30,6,[]
针对今年前5个月全国规模以上工业企业利润数据与去年同期数据的差异问题，国家统计局工业司有关负 ...,20180629,29,6,[]
今日，内蒙古自治区住房资金管理中心网站发布《关于进一步规范调整住房公积金提取政策的通知》称， ...,20180702,2,7,[]
自2016年10月份国务院发布《关于市场化银行债权转股权的指导意见》以来，债转股一直存在“落 ...,20180702,2,7,[银行]
污水处理、垃圾处理、节水、节能环保……国家发展改革委2日对外发布《关于创新和完善促进绿色发展 ...,20180702,2,7,[]
29日，中国人民银行会同银保监会、证监会、国家发展改革委、财政部等联合召开全国深化小微企业金 ...,20180630,30,6,[银行]
货币政策委员会二季度例会释放信号——日前召开的中国人民银行货币政策委员会2018年第二季度例 ...,20180702,2,7,[银行]
近日，由中国旅游研究院主办，人民日报客户端旅游频道提供媒体支持的《中国入境旅游发展年度报告2 ...,20180702,2,7,[]
作为银行发起设立的债转股实施机构，金融资产投资公司是一类新型实施机构。从昨日开始，这类机构有 ...,20180630,30,6,[银行]


# 检查数据纯净度
* 测试用，检查清洗数据的效果

In [5]:
# 随机人工检查数据纯净度

def showContents(number_to_show, total_amount):
    counter = 0
    while counter < number_to_show:
        print(news['contents'][int(random.uniform(0, total_amount - 1))])
        print('')
        counter += 1
    
showContents(3, len(news['contents']))

6月29日，港铁邀请发展商及财团提交黄竹坑站第三期物业发展项目的标书，于8月7日下午两点正截标。据悉，港铁于6月21日起就黄竹坑站第三期物业发展项目，邀请有兴趣的发展商及财团提交发展意向书。截至6月27日下午2时，港铁发言人称，项目共收到36份意向书。中港发展商皆有参与投交意向书，包括中信泰富、帝国集团、会德丰、华懋、鹰君、英皇国际、新世界发展、嘉里建设、爪哇控股、建灏地产、庄士机构国际、高银地产、泛海国际、南丰发展、远东发展、中海外及碧桂园等。据观点地产新媒体获悉，港铁黄竹坑站第三期属于黄竹坑站上盖唯一的商住发展项目，总楼面面积约150.6万平方呎，当中近100万平方呎属于住宅楼面，将会兴建4幢住宅楼宇，提供住宅约1200伙，并设一个50.6万平方呎的商场。项目地价市值达225.9亿至361.4亿港元，呎价达1.5万至2.4万港元，预计将于2024年落成。

为全面介绍中国履行加入世界贸易组织承诺的实践，阐释中国参与多边贸易体制建设的原则立场和政策主张，阐明中国推进更高水平对外开放的愿景与行动，中国政府日前发表《中国与世界贸易组织》白皮书。这是中国首次就这一问题发表白皮书，引发国际社会广泛关注。海外人士高度评价中国入世以来对世界贸易所作贡献。世界贸易组织发言人基思·罗克韦尔对新华社记者说，世界贸易组织秘书处过去和现在都“十分赞赏中国对世界贸易组织所给予的强有力支持”。罗克韦尔说，中国一直积极参与世贸组织各个领域的活动，“很清楚的是，中国的参与为《贸易便利化协定》谈判和诸边性《信息技术协定》扩容谈判的成功都发挥了至关重要的作用”。俄罗斯国民经济和国家行政学院副教授谢尔盖·海斯坦诺夫说，中国履行了所有作为世贸组织成员的义务。从世界贸易角度看，中国加入世贸组织给全球贸易发展注入动力。俄高等经济学院东方学教研室主任阿列克谢·马斯洛夫说，中国加入世贸组织后，世界贸易往来变得更加活跃，全球贸易额逐步增加，中国与全球各国间的贸易额均呈增长之势。马斯洛夫还认为，中国入世使国际市场的良性竞争增强。中国电子产品和其他科技产品具有竞争优势，这促使海外企业不断通过提升产品质量与中国企业展开良性竞争。俄科学院远东研究所副所长、中国问题专家安德烈·奥斯特洛夫斯基表示，中国切实履行了入世承诺。中国外贸额自2001年以来大幅增长，世界贸易因此繁荣。中国一直推动多边贸易发展，尤其是“一带一路”倡议

# 自然语言识别

## 第一步 分句分词

### 分句函数
* 将句子分开并创建名为'content_split_sents'的列

In [6]:
# 分句函数
def splitSents(contents):
    return SentenceSplitter.split(contents)

# 执行分句函数
news['content_split_sents'] = news['contents'].apply(splitSents)

# 预览
news

contents,date,day,month,related
小微企业融资难、融资贵问题，是一个世界性难题。甚至，外延更为丰富的普惠金融，在很多时候、在不 ...,20180630,30,6,"[保险, 陆家嘴, 银行]"
据国家统计局30日消息，2018年6月份，中国制造业采购经理指数(PMI)为51.5%，比上 ...,20180630,30,6,[]
针对今年前5个月全国规模以上工业企业利润数据与去年同期数据的差异问题，国家统计局工业司有关负 ...,20180629,29,6,[]
今日，内蒙古自治区住房资金管理中心网站发布《关于进一步规范调整住房公积金提取政策的通知》称， ...,20180702,2,7,[]
自2016年10月份国务院发布《关于市场化银行债权转股权的指导意见》以来，债转股一直存在“落 ...,20180702,2,7,[银行]
污水处理、垃圾处理、节水、节能环保……国家发展改革委2日对外发布《关于创新和完善促进绿色发展 ...,20180702,2,7,[]
29日，中国人民银行会同银保监会、证监会、国家发展改革委、财政部等联合召开全国深化小微企业金 ...,20180630,30,6,[银行]
货币政策委员会二季度例会释放信号——日前召开的中国人民银行货币政策委员会2018年第二季度例 ...,20180702,2,7,[银行]
近日，由中国旅游研究院主办，人民日报客户端旅游频道提供媒体支持的《中国入境旅游发展年度报告2 ...,20180702,2,7,[]
作为银行发起设立的债转股实施机构，金融资产投资公司是一类新型实施机构。从昨日开始，这类机构有 ...,20180630,30,6,[银行]


## 移除标点符号

In [83]:
# 定义移除函数
def removePunctuation(contents):
    return re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "", contents)

<input>:3: DeprecationWarning: invalid escape sequence \s
<input>:3: DeprecationWarning: invalid escape sequence \s
<input>:3: DeprecationWarning: invalid escape sequence \s
<ipython-input-83-e29b267f43ac>:3: DeprecationWarning: invalid escape sequence \s
  return re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "", contents)


In [80]:
# 运行移除函数并创建一个名为'contents_nopunc'的列
news['contents_nopunc'] = news['contents'].apply(removePunctuation)

<unknown>:2: DeprecationWarning: invalid escape sequence \s


In [85]:
# 预览
news['contents_nopunc'][2]

'针对今年前5个月全国规模以上工业企业利润数据与去年同期数据的差异问题国家统计局工业司有关负责人29日回应称这一数据是真实可靠实际可比的数据相比略有差异主要原因在于企业调查范围变化剔除重复计算和非工业生产经营活动剥离日前国家统计局发布数据2018年1至5月份全国规模以上工业企业实现利润总额272983亿元同比增长165去年同期国家统计局公布的数据显示2017年1至5月份全国规模以上工业企业实现利润总额290476亿元同比增长227简单对比显示今年前5个月的利润数值比去年减少为何数据显示仍同比增长国家统计局说明了三方面原因：一是加强统计执法进一步规范了规模以上工业企业统计范围按照中央建立高质量统计体系要求国家统计局对不符合规模以上工业统计要求的企业进行了剔除此外根据制度规定每年定期要对规模以上工业企业调查范围进行调整每年有部分企业达到规模标准纳入调查范围也有部分企业因规模较小而退出调查范围还有新建投产企业注吊)销企业等变化带来数据略有减小二是加强数据质量管理剔除跨地区跨行业重复统计数据近年来企业跨地区跨行业经营越来越普遍国家统计局统计执法检查发现部分企业违反统计制度将下属跨地区法人企业包含在本企业当中进行统计造成同一法人在不同地区间和不同行业间被重复统计根据国家统计局最新开展的企业组织结构调查情况去年四季度开始对企业集团公司)跨地区跨行业重复计算进行了剔重三是实施“营改增”政策后一些非工业生产经营活动剥离在实施“营改增”政策前部分工业企业将本应缴纳营业税的非工业生产经营活动纳入本企业的财务核算用以抵扣销项税在“营改增”政策实施后服务业企业改交增值税且税率较低工业企业逐步将内部非工业生产经营活动剥离转向服务业也使工业企业财务数据有所减小国家统计局工业司有关负责人说应当看到国家统计局发布的规模以上工业企业利润总额与财政部的国有企业数据和上市公司财务季报数据总体趋势一致是反映企业实际情况的'

### 分词函数
* 将词分开 并创建名为'contents_split_words'的列

In [7]:
# 分词函数
def splitWords(contents):
    segmentor = Segmentor()  # 初始化实例
    segmentor.load(cws_model_path)  # 加载模型
    words = segmentor.segment(contents)  # 分词
    segmentor.release()  # 释放模型
    return words

# 执行分词函数
news['content_split_words'] = news['contents'].apply(splitWords)

# 预览
news

contents,date,day,month,related
小微企业融资难、融资贵问题，是一个世界性难题。甚至，外延更为丰富的普惠金融，在很多时候、在不 ...,20180630,30,6,"[保险, 陆家嘴, 银行]"
据国家统计局30日消息，2018年6月份，中国制造业采购经理指数(PMI)为51.5%，比上 ...,20180630,30,6,[]
针对今年前5个月全国规模以上工业企业利润数据与去年同期数据的差异问题，国家统计局工业司有关负 ...,20180629,29,6,[]
今日，内蒙古自治区住房资金管理中心网站发布《关于进一步规范调整住房公积金提取政策的通知》称， ...,20180702,2,7,[]
自2016年10月份国务院发布《关于市场化银行债权转股权的指导意见》以来，债转股一直存在“落 ...,20180702,2,7,[银行]
污水处理、垃圾处理、节水、节能环保……国家发展改革委2日对外发布《关于创新和完善促进绿色发展 ...,20180702,2,7,[]
29日，中国人民银行会同银保监会、证监会、国家发展改革委、财政部等联合召开全国深化小微企业金 ...,20180630,30,6,[银行]
货币政策委员会二季度例会释放信号——日前召开的中国人民银行货币政策委员会2018年第二季度例 ...,20180702,2,7,[银行]
近日，由中国旅游研究院主办，人民日报客户端旅游频道提供媒体支持的《中国入境旅游发展年度报告2 ...,20180702,2,7,[]
作为银行发起设立的债转股实施机构，金融资产投资公司是一类新型实施机构。从昨日开始，这类机构有 ...,20180630,30,6,[银行]


## 第二步 词性标注

### 词性分别标注

In [8]:
def tagWords(contents_split):
    postagger = Postagger() # 初始化实例
    postagger.load(pos_model_path)  # 加载模型

    words = contents_split  # 分词结果
    postags = postagger.postag(words)  # 词性标注

    postagger.release()  # 释放模型
    return postags

In [9]:
# 执行分词函数
news['content_tag_words'] = news['content_split_words'].apply(tagWords)

# 预览
news

contents,date,day,month,related
小微企业融资难、融资贵问题，是一个世界性难题。甚至，外延更为丰富的普惠金融，在很多时候、在不 ...,20180630,30,6,"[保险, 陆家嘴, 银行]"
据国家统计局30日消息，2018年6月份，中国制造业采购经理指数(PMI)为51.5%，比上 ...,20180630,30,6,[]
针对今年前5个月全国规模以上工业企业利润数据与去年同期数据的差异问题，国家统计局工业司有关负 ...,20180629,29,6,[]
今日，内蒙古自治区住房资金管理中心网站发布《关于进一步规范调整住房公积金提取政策的通知》称， ...,20180702,2,7,[]
自2016年10月份国务院发布《关于市场化银行债权转股权的指导意见》以来，债转股一直存在“落 ...,20180702,2,7,[银行]
污水处理、垃圾处理、节水、节能环保……国家发展改革委2日对外发布《关于创新和完善促进绿色发展 ...,20180702,2,7,[]
29日，中国人民银行会同银保监会、证监会、国家发展改革委、财政部等联合召开全国深化小微企业金 ...,20180630,30,6,[银行]
货币政策委员会二季度例会释放信号——日前召开的中国人民银行货币政策委员会2018年第二季度例 ...,20180702,2,7,[银行]
近日，由中国旅游研究院主办，人民日报客户端旅游频道提供媒体支持的《中国入境旅游发展年度报告2 ...,20180702,2,7,[]
作为银行发起设立的债转股实施机构，金融资产投资公司是一类新型实施机构。从昨日开始，这类机构有 ...,20180630,30,6,[银行]


## 第三步 命名实体识别

In [13]:
# 爆破实体识别
recognized_words = []
counter = 0
while counter < len(news['title']):
    recognizer = NamedEntityRecognizer() # 初始化实例
    recognizer.load(ner_model_path)  # 加载模型
    netags = recognizer.recognize(news['content_split_words'][counter], news['content_tag_words'][counter])  # 命名实体识别
    recognizer.release()  # 释放模型
    recognized_words.append(netags)
    counter += 1

In [14]:
news['recognized_words'] = recognized_words
print(len(news['recognized_words']))

509


In [16]:
news

contents,date,day,month,related
小微企业融资难、融资贵问题，是一个世界性难题。甚至，外延更为丰富的普惠金融，在很多时候、在不 ...,20180630,30,6,"[保险, 陆家嘴, 银行]"
据国家统计局30日消息，2018年6月份，中国制造业采购经理指数(PMI)为51.5%，比上 ...,20180630,30,6,[]
针对今年前5个月全国规模以上工业企业利润数据与去年同期数据的差异问题，国家统计局工业司有关负 ...,20180629,29,6,[]
今日，内蒙古自治区住房资金管理中心网站发布《关于进一步规范调整住房公积金提取政策的通知》称， ...,20180702,2,7,[]
自2016年10月份国务院发布《关于市场化银行债权转股权的指导意见》以来，债转股一直存在“落 ...,20180702,2,7,[银行]
污水处理、垃圾处理、节水、节能环保……国家发展改革委2日对外发布《关于创新和完善促进绿色发展 ...,20180702,2,7,[]
29日，中国人民银行会同银保监会、证监会、国家发展改革委、财政部等联合召开全国深化小微企业金 ...,20180630,30,6,[银行]
货币政策委员会二季度例会释放信号——日前召开的中国人民银行货币政策委员会2018年第二季度例 ...,20180702,2,7,[银行]
近日，由中国旅游研究院主办，人民日报客户端旅游频道提供媒体支持的《中国入境旅游发展年度报告2 ...,20180702,2,7,[]
作为银行发起设立的债转股实施机构，金融资产投资公司是一类新型实施机构。从昨日开始，这类机构有 ...,20180630,30,6,[银行]


## 第四步 生成Word Count Vector

In [25]:
# 生成Word Count Vector
news['word_count'] = tc.text_analytics.count_words(news['content_split_words'])

In [26]:
news

contents,date,day,month,related
小微企业融资难、融资贵问题，是一个世界性难题。甚至，外延更为丰富的普惠金融，在很多时候、在不 ...,20180630,30,6,"[保险, 陆家嘴, 银行]"
据国家统计局30日消息，2018年6月份，中国制造业采购经理指数(PMI)为51.5%，比上 ...,20180630,30,6,[]
针对今年前5个月全国规模以上工业企业利润数据与去年同期数据的差异问题，国家统计局工业司有关负 ...,20180629,29,6,[]
今日，内蒙古自治区住房资金管理中心网站发布《关于进一步规范调整住房公积金提取政策的通知》称， ...,20180702,2,7,[]
自2016年10月份国务院发布《关于市场化银行债权转股权的指导意见》以来，债转股一直存在“落 ...,20180702,2,7,[银行]
污水处理、垃圾处理、节水、节能环保……国家发展改革委2日对外发布《关于创新和完善促进绿色发展 ...,20180702,2,7,[]
29日，中国人民银行会同银保监会、证监会、国家发展改革委、财政部等联合召开全国深化小微企业金 ...,20180630,30,6,[银行]
货币政策委员会二季度例会释放信号——日前召开的中国人民银行货币政策委员会2018年第二季度例 ...,20180702,2,7,[银行]
近日，由中国旅游研究院主办，人民日报客户端旅游频道提供媒体支持的《中国入境旅游发展年度报告2 ...,20180702,2,7,[]
作为银行发起设立的债转股实施机构，金融资产投资公司是一类新型实施机构。从昨日开始，这类机构有 ...,20180630,30,6,[银行]


In [28]:
news['word_count'][0]

{'学院': 1,
 '兰州': 1,
 '研究院': 1,
 '作者': 1,
 '副作用': 1,
 '带来': 1,
 '空间': 1,
 '另': 1,
 '结构性': 1,
 '使用': 1,
 '过量': 1,
 '边际': 1,
 '撰文': 1,
 '助理': 1,
 '1.75万亿': 1,
 '释放': 1,
 '累计': 1,
 '值得': 1,
 '也': 1,
 '出清': 1,
 '破产': 1,
 '关闭': 1,
 '壮士': 1,
 '拿': 1,
 '输血': 1,
 '掐': 1,
 '结构': 2,
 '且': 1,
 '行业': 1,
 '僵尸': 3,
 '减法': 1,
 '做好': 1,
 '还要': 1,
 '不仅': 1,
 '惠及': 1,
 '行为': 1,
 '违法': 1,
 '补': 1,
 '骗': 1,
 '骗贷': 1,
 '弄虚作假': 1,
 '内外': 1,
 '查处': 1,
 '依': 1,
 '依法': 1,
 '此外': 1,
 '设施': 1,
 '主业': 1,
 '功能': 2,
 '分险': 1,
 '增信': 1,
 '重阳': 1,
 '行动': 1,
 '专项': 1,
 '账款': 1,
 '应': 4,
 '手段': 1,
 '科技': 1,
 '运用': 1,
 '尽职': 1,
 '落实': 1,
 '担忧': 1,
 '自身': 1,
 '征对性': 1,
 '办法': 1,
 '如果': 1,
 '民间': 1,
 '或': 1,
 '超过': 1,
 '收': 1,
 '相对': 1,
 '规': 1,
 '目前': 1,
 '股权': 1,
 '区域性': 1,
 '三高': 1,
 '规范': 1,
 '披露': 1,
 '发行': 1,
 '加': 1,
 '三板': 1,
 '退出': 3,
 '天使': 2,
 '投资': 4,
 '注重': 2,
 '创业': 2,
 '补贴': 1,
 '方式': 3,
 '间接': 1,
 '第四': 1,
 '主体': 1,
 '重心': 1,
 '法人': 1,
 '地方性': 1,
 '成长性': 1,
 '民营': 1,
 '设立': 2,
 '未': 1,
 '网点': 1,
 '下沉': 1,
 

## 第五步 依存句法分析

In [19]:
# -*- coding: utf-8 -*-
parser = Parser() # 初始化实例
parser.load(par_model_path)  # 加载模型

words = ['元芳', '你', '怎么', '看']
postags = ['nh', 'r', 'r', 'v']
arcs = parser.parse(words, postags)  # 句法分析

print("\t".join("%d:%s" % (arc.head, arc.relation) for arc in arcs))
parser.release()  # 释放模型

4:SBV	4:SBV	4:ADV	0:HED


## 第六步 语义角色分析

# 保存数据

In [29]:
# 保存数据
filepath = '../DataSets/Eastmoney/News_NLP/China/'
news.save(filepath + 'NLP' + news_source, format='csv')
# news.save('../DataSets/Eastmoney/News_NLP/China/NLPCHINA20180702-1906.csv', format='csv')

# 打印时间戳
print('程序运行时间戳：20' 
      + str(datetime.datetime.now().strftime("%y")) + '年'
      + str(datetime.datetime.now().strftime("%m")) + '月' 
      + str(datetime.datetime.now().strftime("%d")) + '日' 
      + str(datetime.datetime.now().strftime("%H")) + '时' 
      + str(datetime.datetime.now().strftime("%M")) + '分' 
      + str(datetime.datetime.now().strftime("%S")) + '秒')

# 打印数据路径
print('\n成功保存自然语言识别数据文件！数据路径：')
print(filepath + 'NLP' + news_source)

程序运行时间戳：2018年07月03日20时31分27秒

成功保存自然语言识别数据文件！数据路径：
../DataSets/Eastmoney/News_NLP/China/NLPCHINA20180702-1906.csv
